In [32]:
%pylab inline
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
from itertools import product
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error 
from statsmodels.tsa.holtwinters import ExponentialSmoothing

Populating the interactive namespace from numpy and matplotlib


In [42]:
wine = pd.read_csv('monthly-australian-wine-sales.csv',',', index_col=['month'], parse_dates=['month'], dayfirst=True)

Как работает деление:

In [44]:
X = np.array([100,200,300,400,500,600])

tscv = TimeSeriesSplit()
TimeSeriesSplit(max_train_size=None, n_splits=5) #n_splits - количество фолдов для трейна

for train_index, test_index in tscv.split(X):
    print("TRAIN position:", train_index, "TEST position:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    print("TRAIN value:", X_train, "TEST value:", X_test)

TRAIN position: [0] TEST position: [1]
TRAIN value: [100] TEST value: [200]
TRAIN position: [0 1] TEST position: [2]
TRAIN value: [100 200] TEST value: [300]
TRAIN position: [0 1 2] TEST position: [3]
TRAIN value: [100 200 300] TEST value: [400]
TRAIN position: [0 1 2 3] TEST position: [4]
TRAIN value: [100 200 300 400] TEST value: [500]
TRAIN position: [0 1 2 3 4] TEST position: [5]
TRAIN value: [100 200 300 400 500] TEST value: [600]


Расчет на реальных данных

In [ ]:
#разбиваем на 3 фолда данные
#строим модель отдельно на каждом наборе данных
#оцениваем на каждой ошибку и выдаем среднее значение ошибка

In [62]:
def timeseriesCVscore(series, seasonal_periods=12):
    
    errors = []
    values = series.values
    
    # set the number of folds for cross-validation
    tscv = TimeSeriesSplit(n_splits=3) 
    
    # iterating over folds, train model on each, forecast and calculate error
    for train, test in tscv.split(values):
        model = ExponentialSmoothing(values, seasonal='mul', trend=None, seasonal_periods=seasonal_periods).fit(use_boxcox=True)
        
        predictions = model.predict(start=test[0], end=test[-1])
        actual = values[test]
        error = sqrt(mean_squared_error(predictions, actual))
        errors.append(error)
        
    return np.mean(np.array(errors))

In [63]:
timeseriesCVscore(wine.sales)

2394.754625026853